## Install Packages

In [7]:
import tensorflow as tf


In [8]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Input, Concatenate
from tensorflow.keras.models import Model

Encoder Structure and Decoder Structure

In [13]:
def encoder(input_image):
    x = Conv2D(64, (3,3),activation = "relu", padding = "same")(input_image)
    x = MaxPooling2D((2,2), padding = "same")(x)

    x = Conv2D(128, (3,3),activation = "relu", padding = "same")(x)
    x = MaxPooling2D((2,2), padding = "same")(x)

    x = Conv2D(256 , (3,3),activation = "relu", padding = "same")(x)
    x = MaxPooling2D((2,2), padding = "same")(x)

    return x

def decoder (encoder_layer):
    x = UpSampling2D((2,2))(encoder_layer)
    x = Conv2D(128, (3,3), activation = "relu", padding = "same")(x)

    x = UpSampling2D((2,2))(x)
    x = Conv2D(64, (3,3), activation = "relu", padding = "same")(x)

    x = UpSampling2D((2,2))(x)
    x = Conv2D(32, (3,3), activation = "relu", padding = "same")(x)

    x = Conv2D(16, (3,3), activation = "relu", padding = "same")(x)

    output =  Conv2D(1, (1, 1), activation='sigmoid', padding='same')(x)

    return output 



Build Model


In [14]:
def build_model(input_shape):
    inputs = Input(shape=input_shape)
    encoded_layer = encoder(inputs)
    decoded_layer = decoder(encoded_layer)

    model = Model(inputs, decoded_layer)
    return model

input_shape = (256, 256, 3)  # Example input shape (height, width, channels)
model = build_model(input_shape)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 256, 256, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 128, 128, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 64, 64, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 64, 64, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 32, 32, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_1 (UpSampling2D)  │ (None, 64, 64, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 64, 64, 128)    │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_2 (UpSampling2D)  │ (None, 128, 128, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 128, 128, 64)   │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_3 (UpSampling2D)  │ (None, 256, 256, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 256, 256, 32)   │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 256, 256, 16)   │         4,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 256, 256, 1)    │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 762,753 (2.91 MB)

 Trainable params: 762,753 (2.91 MB)

 Non-trainable params: 0 (0.00 B)

## Compile the Model

In [16]:
def f1_score(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    f1 = 2 * ((precision * recall) / (precision + recall + K.epsilon()))
    return f1

F1score = f1_score(y_true, y_pred)

model.compile(optimizer = "adam", loss = "binary_crossentropy" , metrics = [F1score])

NameError: name 'F1score' is not defined

## Load Dataset


In [ ]:
def load_data():
    # Load your dataset here
    train_images, train_masks = [], []
    val_images, val_masks = [], []
    return (train_images, train_masks), (val_images, val_masks)

(train_images, train_masks), (val_images, val_masks) = load_data()

# Convert to TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_masks))
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_masks))

# Batch and prefetch data
batch_size = 16
train_dataset = train_dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)


## Train The Model

In [ ]:
# Train the model
epochs = 50
history = model.fit(train_dataset, 
                    epochs=epochs, 
                    validation_data=val_dataset)

# Evaluate the model
val_loss, val_f1_score = model.evaluate(val_dataset)
print(f"Validation Loss: {val_loss}, Validation F1 Score: {val_f1_score}")

# Save the model
model.save('human_parsing_model.h5')